# Constants and Libraries

In [1]:
import os, requests, time
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv

if not load_dotenv("./../../../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")

direct_line_secret = os.environ["direct_line_secret"]
sender_id          = "mauromi"
sender_name        = "Mauro Minella"
question           = "my printer is broken"

Environment variables have been loaded ;-)


# Start a new conversation

In [2]:
def new_conversation(direct_line_secret: str):
    # Start a new conversation
    
    headers = {
        'Authorization': f'Bearer {direct_line_secret}',
        'Content-Type': 'application/json'
    }
    start_conv = requests.post(
        'https://directline.botframework.com/v3/directline/conversations',
        headers=headers
    )
    
    conversation_id = start_conv.json()['conversationId']
    return conversation_id


conversation_id = new_conversation(direct_line_secret = direct_line_secret)

conversation_id

'4dGFCmrgcF57ljn7UIAtOj-fr'

# Send a message to the bot

In [3]:
def send_message_to_bot(conversation_id: str, question: str, sender_id: str="user", sender_name: str=""):

    if sender_name=="":
        sender_name = sender_id

    message = {
        'type': 'message',
        'from': {'id': sender_id, 'name': sender_name},
        'text': question
    }

    headers = {
        'Authorization': f'Bearer {direct_line_secret}',
        'Content-Type': 'application/json'
    }
    
    response = requests.post(
        f'https://directline.botframework.com/v3/directline/conversations/{conversation_id}/activities',
        headers=headers,
        json=message,
    )
    return response.json();


send_message_to_bot(conversation_id=conversation_id, question=question, sender_id=sender_id)

{'id': '4dGFCmrgcF57ljn7UIAtOj-fr|0000000'}

# Poll the activities endpoint

In [4]:
def print_conversation(conversation_id: str, last_question: str):
    activities_url = f"https://directline.botframework.com/v3/directline/conversations/{conversation_id}/activities"
    headers = {
        'Authorization': f'Bearer {direct_line_secret}',
        'Content-Type': 'application/json'
    }
    
    while True:
        response = requests.get(activities_url, headers=headers)
        activities = response.json()['activities']
        if ((activities[-1]["type"] == "message" and activities[-1]["text"] != last_question)
            or (activities[-1]["type"] == "event" and activities[-1]["valueType"] == "DynamicPlanFinished")):
            if (activities[-1]["type"] == "event"):
                print(f"Status: {activities[-1]['valueType']}. FINISHED!\n\n")
            else:
                print("FINISHED!\n\n")
            break
        else:
            time.sleep(1)
            if (activities[-1]["type"] == "event"):
                print(f"Status: {activities[-1]['valueType']}. Waiting for answer to be completed...")
            else:
                print("Waiting for answer to be completed...")

    i = 0

    for activity in activities:
        if activity["type"] == "message":
            i += 1
            display(Markdown(f'Message {i} - {activity["from"]["name"]} says: {activity["text"]}\n'))

    activities = [activity for activity in activities if activity.get('type') == 'message']

    return activities

activities = print_conversation(conversation_id=conversation_id, last_question=question)

Waiting for answer to be completed...
Waiting for answer to be completed...
Waiting for answer to be completed...
Status: DynamicPlanStepBindUpdate. Waiting for answer to be completed...
Status: DynamicPlanStepBindUpdate. Waiting for answer to be completed...
Status: DynamicPlanStepBindUpdate. Waiting for answer to be completed...
Status: DynamicPlanStepBindUpdate. Waiting for answer to be completed...
Status: DynamicPlanStepFinished. Waiting for answer to be completed...
Status: DynamicPlanStepFinished. Waiting for answer to be completed...
Status: DynamicPlanStepFinished. Waiting for answer to be completed...
Status: DynamicPlanFinished. FINISHED!




Message 1 - mauromi says: my printer is broken


Message 2 - Assistente Onboarding says: Here are some general troubleshooting steps for a broken printer:

1. **Check the Basics**:
   - Ensure the printer is plugged in and turned on.
   - Verify that all cables are securely connected.
   - Make sure there is paper in the tray and that it is loaded correctly.

2. **Check for Error Messages**:
   - Look at the printer's display panel for any error messages or blinking lights. Refer to the printer's manual to understand what these messages mean.

3. **Restart the Printer**:
   - Turn off the printer, wait for a few seconds, and then turn it back on. Sometimes a simple restart can resolve the issue.

4. **Check the Ink or Toner Levels**:
   - Ensure that the printer has enough ink or toner. Replace any empty cartridges if necessary.

5. **Clear Paper Jams**:
   - Open the printer and check for any paper jams. Carefully remove any stuck paper, following the instructions in the printer's manual.

6. **Update or Reinstall Drivers**:
   - Ensure that the printer drivers are up to date. You can download the latest drivers from the printer manufacturer's website.
   - If necessary, uninstall the current drivers and reinstall them.

7. **Check Printer Settings**:
   - Make sure the printer is set as the default printer on your computer.
   - Verify that the printer is not set to offline mode.

8. **Run Printer Troubleshooter**:
   - Many operating systems have built-in printer troubleshooting tools. Run the troubleshooter to identify and fix common issues.

9. **Check Network Connections** (for wireless printers):
   - Ensure that the printer is connected to the correct Wi-Fi network.
   - Restart your router if necessary.

10. **Consult the Manual or Manufacturer Support**:
   - If the problem persists, refer to the printer's manual for more specific troubleshooting steps.
   - Contact the printer manufacturer's customer support for further assistance.

If you need more detailed instructions or have a specific issue, feel free to ask!


In [5]:
activities

[{'type': 'message',
  'id': '4dGFCmrgcF57ljn7UIAtOj-fr|0000000',
  'timestamp': '2025-06-26T23:43:01.5084308Z',
  'serviceUrl': 'https://directline.botframework.com/',
  'channelId': 'directline',
  'from': {'id': 'mauromi', 'name': 'mauromi'},
  'conversation': {'id': '4dGFCmrgcF57ljn7UIAtOj-fr'},
  'text': 'my printer is broken'},
 {'type': 'message',
  'id': '4dGFCmrgcF57ljn7UIAtOj-fr|0000007',
  'timestamp': '2025-06-26T23:43:13.5715591Z',
  'channelId': 'directline',
  'from': {'id': '1487de36-6cf4-98db-896b-dd501b2d01a0',
   'name': 'Assistente Onboarding',
   'role': 'bot'},
  'conversation': {'id': '4dGFCmrgcF57ljn7UIAtOj-fr'},
  'textFormat': 'markdown',
  'membersAdded': [],
  'membersRemoved': [],
  'reactionsAdded': [],
  'reactionsRemoved': [],
  'text': "Here are some general troubleshooting steps for a broken printer:\n\n1. **Check the Basics**:\n   - Ensure the printer is plugged in and turned on.\n   - Verify that all cables are securely connected.\n   - Make sure the

# All in one

In [6]:
question="I need a plan for my next vacation in Australia"
conversation_id = new_conversation(direct_line_secret = direct_line_secret)
send_message_to_bot(conversation_id=conversation_id, question=question, sender_id=sender_id)
activities = print_conversation(conversation_id=conversation_id, last_question=question)

Waiting for answer to be completed...
Waiting for answer to be completed...
Status: DynamicPlanReceived. Waiting for answer to be completed...
Status: DynamicPlanStepBindUpdate. Waiting for answer to be completed...
Status: DynamicPlanStepBindUpdate. Waiting for answer to be completed...
Status: DynamicPlanStepBindUpdate. Waiting for answer to be completed...
Status: DynamicPlanStepBindUpdate. Waiting for answer to be completed...
Status: DynamicPlanStepFinished. Waiting for answer to be completed...
Status: DynamicPlanStepFinished. Waiting for answer to be completed...
Status: DynamicPlanStepFinished. Waiting for answer to be completed...
Status: DynamicPlanStepFinished. Waiting for answer to be completed...
Status: DynamicPlanFinished. FINISHED!




Message 1 - mauromi says: I need a plan for my next vacation in Australia


Message 2 - Assistente Onboarding says: Planning a vacation in Australia can be an exciting adventure! Here are some steps to help you get started:

### 1. **Decide on the Duration**
Determine how long you plan to stay in Australia. This will help you plan your itinerary and budget.

### 2. **Choose Your Destinations**
Australia is vast and diverse. Here are some popular destinations:
- **Sydney**: Famous for the Sydney Opera House, Harbour Bridge, and beautiful beaches.
- **Melbourne**: Known for its vibrant arts scene, coffee culture, and laneways.
- **Great Barrier Reef**: A must-see for snorkeling and diving enthusiasts.
- **Uluru (Ayers Rock)**: Experience the iconic red rock and learn about Aboriginal culture.
- **Gold Coast**: Perfect for surfing, theme parks, and nightlife.

### 3. **Book Flights and Accommodation**
- **Flights**: Look for deals on flights to Australia. Major cities like Sydney and Melbourne have international airports.
- **Accommodation**: Choose from hotels, hostels, Airbnb, or vacation rentals depending on your budget and preferences.

### 4. **Plan Activities**
- **Outdoor Adventures**: Hiking, surfing, diving, and exploring national parks.
- **Cultural Experiences**: Visit museums, galleries, and historical sites.
- **Wildlife Encounters**: See kangaroos, koalas, and other unique Australian animals.

### 5. **Transportation**
- **Car Rental**: Renting a car can give you the freedom to explore at your own pace.
- **Public Transport**: Cities like Sydney and Melbourne have efficient public transport systems.

### 6. **Prepare Travel Documents**
- **Visa**: Check visa requirements for your country and apply in advance.
- **Travel Insurance**: Consider getting travel insurance for peace of mind.

### 7. **Pack Accordingly**
- **Clothing**: Pack for the season and activities you plan to do. Australia’s climate varies greatly.
- **Essentials**: Sunscreen, hat, comfortable shoes, and a camera.

### 8. **Health and Safety**
- **Vaccinations**: Check if any vaccinations are required.
- **Emergency Contacts**: Have a list of emergency contacts and local health facilities.

### 9. **Budgeting**
- **Expenses**: Plan your budget for accommodation, food, activities, and transportation.
- **Currency**: The local currency is the Australian Dollar (AUD).

### 10. **Local Etiquette**
- **Respect Nature**: Follow guidelines in national parks and beaches.
- **Cultural Sensitivity**: Be respectful of Aboriginal culture and traditions.

Enjoy your trip to Australia! If you need more specific information or help with booking, feel free to ask.
